In [1]:
import sys 
import os
import subprocess
import numpy as np 
import time
from itertools import product 
import glob
import re
import pandas as pd
import pysynapse
from ddot import Ontology
import multiprocessing as mp
from itertools import chain
import copy


output_dir = '../output/analyzing_clixo_ontologies'; 
if not os.path.exists(output_dir): os.makedirs(output_dir)

In [2]:
def get_alpha_beta(l):
    """
    Yield function to get the correct alpha and beta names.
    
    """
    for x in l: 
        alpha = '{:.1f}'.format(x[0])
        beta =  '{:.1f}'.format(x[1])
        yield((alpha, beta))
        
    yield(None, None)

def read_clixo_stats(fn):
    dic = {'num_valid_clusters': np.nan, 'largest_cluster': np.nan, 
          'num_edges_in_clustergraph': np.nan, 'num_clusters': np.nan}
    with open(fn) as f: 
        for line in f: 
            if line.startswith('#'):
                if 'Num valid clusters' in line: 
                    dic['num_valid_clusters'] = int(line.strip().split()[-1])
                elif 'Largest cluster' in line:
                    dic['largest_cluster'] = int(line.strip().split()[-1])
                elif 'Num edges in clusterGraph' in line:
                    dic['num_edges_in_clustergraph'] = int(line.strip().split()[-1])
                elif 'Num clusters' in line:
                    dic['num_clusters'] = int(line.strip().split()[-1])

    return(dic)

## Clean the CliXo ontology for Enrichment Analysis 

In [3]:
def generate_clean_ont_file(fn, out_fn): 
    
    with open(fn) as fr, open(out_fn, 'w') as fw: 
        
        fw.write('Parent\tChild\tEdgeType\n')
        for line in fr:
            if line.startswith('#'):
                continue
            else:
                line = line.strip().split('\t')
                line[2] = line[2].replace('default', 'Child-Parent')
                line[2] = line[2].replace('gene', 'Gene-Term')
                line = '\t'.join(line[0:3]) + '\n'
                fw.write(line)
    return(clean_fn)

In [4]:
pattern = 'option([0-9])'
for fn in sorted(glob.glob('../output/run_clixo/option*/*')):
    
    if 'archive' in fn:
        continue 
        
    option = re.search(pattern, fn).groups()[0]
    clean_fn = 'option{}_{}'.format(option, os.path.basename(fn))
    clean_fn = os.path.join(output_dir, clean_fn)
    if not os.path.exists(clean_fn):
        clean_fn = generate_clean_ont_file(fn, clean_fn) 

## Calculate the number of genes 

In [5]:
def read_number_of_genes(fn):
    genes = set() 
    with open(fn) as f:
        for line in f:
            line = line.split() 
            if line[2] == 'Gene-Term':
                genes.add(line[1]) 
    return(len(genes))

In [6]:
gene_stats = []
pattern = 'option([0-9]).*alpha([0-9]*\.[0-9]*)_beta([0-9]*\.[0-9]*)'
for fn in sorted(glob.glob(os.path.join(output_dir, '*txt'))):
        
    params = re.search(pattern, fn)
    option, alpha, beta = [float(x) for x in params.groups()]  
    
    clean_fn = os.path.basename(fn)
    clean_fn = os.path.join(output_dir, clean_fn)
    
    num_genes = read_number_of_genes(clean_fn)
    gene_stats.append([option, alpha, beta, num_genes])

gene_stats = pd.DataFrame(gene_stats, columns=['option', 'alpha', 'beta', 'num_genes'])
option_grps = gene_stats.groupby('option')

In [7]:
gene_stats[gene_stats.num_genes == 0 ]

,option,alpha,beta,num_genes
287,4.0,0.6,0.5,0
292,4.0,0.7,0.5,0
297,4.0,0.8,0.5,0
302,4.0,0.9,0.5,0
303,4.0,0.9,0.6,0


In [8]:
option_grps.num_genes.describe()

,count,mean,std,min,25%,50%,75%,max
option,,,,,,,,
0.0,1.0,1177.000000,NaN,1177.0,1177.0,1177.0,1177.0,1177.0
2.0,80.0,1283.000000,0.000000,1283.0,1283.0,1283.0,1283.0,1283.0
4.0,226.0,1642.831858,247.653769,0.0,1680.0,1680.0,1680.0,1680.0


In [9]:
option_grps.num_genes.value_counts()

option  num_genes
0.0     1177           1
2.0     1283          80
4.0     1680         221
        0              5
Name: num_genes, dtype: int64

In [10]:
option_grps.get_group(4.0).tail(7)

,option,alpha,beta,num_genes
300,4.0,0.8,0.8,1680
301,4.0,0.8,0.9,1680
302,4.0,0.9,0.5,0
303,4.0,0.9,0.6,0
304,4.0,0.9,0.7,1680
305,4.0,0.9,0.8,1680
306,4.0,0.9,0.9,1680


In [11]:
option_grps.get_group(2.0).tail(7)

,option,alpha,beta,num_genes
74,2.0,0.04,0.43,1283
75,2.0,0.04,0.44,1283
76,2.0,0.04,0.45,1283
77,2.0,0.04,0.46,1283
78,2.0,0.04,0.47,1283
79,2.0,0.04,0.48,1283
80,2.0,0.04,0.49,1283


## Calculate CliXo statistics 

In [14]:
clixo_stats = {}
pattern = 'option([0-9]).*alpha([0-9]*\.[0-9]*)_beta([0-9]*\.[0-9]*)'
for fn in sorted(glob.glob('../output/run_clixo/option*/*')):
        
    if 'archive' in fn:
        continue 
        
    params = re.search(pattern, fn)
    option, alpha, beta = [float(x) for x in params.groups()] 
    option = int(option)
    if alpha == None:
        break

    # Getting CliXo stats 
    stats = read_clixo_stats(fn) 
    
    clean_fn = 'option{}_{}'.format(option, os.path.basename(fn))
    clean_fn = os.path.join(output_dir, clean_fn)
    
    # Getting GO enrichment stats
    with open(clean_fn) as f: 
        file_len = len(f.readlines())
    if file_len > 1: 
        go_enrichment = pysynapse.compare_to_go(clean_fn)
    else:
        go_enrichment = np.nan
    stats['go_enrichment'] = go_enrichment    
    key = tuple([option, alpha, beta]) 
    clixo_stats[key] = stats

In [15]:
clixo_stats_df = pd.DataFrame.from_dict(clixo_stats, orient='index')
clixo_stats_df.reset_index(inplace=True)
clixo_stats_df.rename(columns={'level_0': 'option', 
                               'level_1': 'alpha',
                               'level_2': 'beta'}, inplace=True)

In [16]:
clixo_stats_df.go_enrichment.describe()

count    301.000000
mean      44.245847
std       25.613526
min        0.000000
25%       23.000000
50%       47.000000
75%       64.000000
max      100.000000
Name: go_enrichment, dtype: float64

## Analyze disease enrichment with Multiprocessing 

In [17]:
def analyze_gene_enrichment_test(ont_fn, disease_gene_fn, alpha):

    ## We read in our ontology
    #Generate custom ontology from Chromatin branch from human GO
    #ontology_file=Generate_Ontology_File('GO:0000785')
    ont = Ontology.from_table(ont_fn)
    translated = pysynapse.Find_GO_Focus_GeneDict(ont)

    ont = Ontology.from_table(ont_fn)

    #Test genes: autism
    text_file = open(disease_gene_fn, "r")
    test_gene_list = text_file.read().splitlines()
    text_file.close()

    #print("Number of autism genes in our ontology:" ,  len(set(ont.genes).intersection(set(test_gene_list))))
    num_ont_disease_genes = len(set(ont.genes).intersection(set(test_gene_list)))

    #Find number of test genes in enriched modules:
    num_enriched_disease_genes = \
        pysynapse.Find_num_genes_in_enriched(ont, translated, test_gene_list, alpha=0.3)

    return((num_ont_disease_genes, num_enriched_disease_genes))


In [18]:
diseases = ['adhd', 'autism', 'bipolar', 'mdd', 'schizophrenia']

In [20]:
disease_stats = {}

# Define an output queue
output = mp.Queue()

pattern = 'option([0-9]).*alpha([0-9]*\.[0-9]*)_beta([0-9]*\.[0-9]*)'
clean_fns = glob.glob(os.path.join(output_dir + '/*.txt'))

for clean_fn in sorted(clean_fns):
          
    params = re.search(pattern, clean_fn)
    option, alpha, beta = [float(x) for x in params.groups()]    
    
    with open(clean_fn) as f: 
        file_len = len(f.readlines())
    if file_len > 1: 
        # Setup a list of processes that we want to run
        processes = []
        results = []
        for disease in diseases: 
            disease_gene_fn = '../output/omim_psychiatric_disease_genes/{}.txt'.format(disease)
            result = analyze_gene_enrichment_test(clean_fn, disease_gene_fn, alpha=0.3)
            results.append(result) 
        disease_stats[(option, alpha, beta)] = results 

In [21]:
disease_cols = [('{}_genes'.format(disease), 
                 '{}_enriched_genes'.format(disease)) for disease in diseases]
disease_cols = list(chain(*disease_cols))
disease_cols = ['option', 'alpha', 'beta'] + disease_cols
disease_stats_tmp = [list(k) + list(chain(*v)) for k, v in disease_stats.items()]
disease_stats_df = pd.DataFrame(disease_stats_tmp, columns=disease_cols)

## Network metrics 

In [22]:
#distribution for the breadth of child-parent edges
#Loading clixo data
pattern = 'option([0-9]).*alpha([0-9]*\.[0-9]*)_beta([0-9]*\.[0-9]*)'

network_results = {}
for clean_fn in sorted(clean_fns):
          
    params = re.search(pattern, clean_fn)
    option, alpha, beta = [float(x) for x in params.groups()] 

    clixo_ont = pd.read_table(clean_fn, dtype=str)
    child_dist = clixo_ont[clixo_ont.EdgeType == 'Child-Parent']
    parent_groups = child_dist.groupby('Parent')
    gene_dist = parent_groups.Child.count()
    gene_info = gene_dist.describe()[['mean', 'min', '25%', '50%', '75%', 'max']].tolist()

    #distribution for the breadth of child-parent edges
    module_dist = clixo_ont[clixo_ont.EdgeType == 'Gene-Term']
    parent_groups = module_dist.groupby('Parent')
    module_dist = parent_groups.Child.count()
    module_info = module_dist.describe()[['mean', 'min', '25%', '50%', '75%', 'max']].tolist()
    
    results = gene_info + module_info 
    network_results[(option, alpha, beta)] = results 

network_cols = ['option', 'alpha', 'beta']
network_cols += ['gd_{}'.format(x) for x in ['mean', 'min', '25%',  '50%', '75%', 'max']]
network_cols += ['md_{}'.format(x) for x in ['mean', 'min', '25%',  '50%', '75%', 'max']]
network_results = [list(k) + list(v) for k, v in network_results.items()]
network_df = pd.DataFrame(network_results, columns=network_cols)

## Merge the dataframes together 

In [23]:
merged_df = pd.merge(clixo_stats_df, disease_stats_df, 
                     on=['option', 'alpha', 'beta'], how='left')
merged_df = pd.merge(merged_df, network_df, on=['option', 'alpha', 'beta'])
fn = os.path.join(output_dir, 'ontology_stats.tsv')
merged_df.to_csv(fn, sep='\t', index=False)

## Counting the number of dieased genes in ont

In [24]:
clean_fn = clean_fns[0]
for disease in diseases:

    fn = '../output/omim_psychiatric_disease_genes/{}.txt'.format(disease)
    with open(fn) as f: 
        disease_genes = [x.strip() for x in f.readlines()]
    
    ## We read in our ontology
    #Generate custom ontology from Chromatin branch from human GO
    #ontology_file=Generate_Ontology_File('GO:0000785')
    ont = Ontology.from_table(clean_fn)
    inter = len(set(ont.genes).intersection(set(disease_genes)))
    
    print(disease, inter)

adhd 14
autism 12
bipolar 1
mdd 3
schizophrenia 16
